In [3]:
import math

from ib_insync import *
util.startLoop()

ib = IB()
ib.client.setConnectOptions('+PACEAPI')
ib.connect('127.0.0.1', 7497, clientId=20)

<IB connected to 127.0.0.1:7497 clientId=20>

In [4]:
import numpy as np
import yahoo_fin.stock_info as si
import requests
import pandas as pd
import datetime as dt
import os
import os.path
from dateutil.relativedelta import relativedelta
from py_vollib.black.greeks.analytical import delta
from Scripts.Utils import trading_utils
import math
from dateutil import parser
import yfinance as yf

In [5]:
def FindMinMaxStrike(expiry, atm_volatility, std_to_look_upto, atm_strike):
    time_to_expiry = trading_utils.tte(dt.datetime.now(), parser.parse(expiry))
    variance_till_expiry = atm_volatility * atm_volatility * time_to_expiry
    vol_till_expiry = math.sqrt(variance_till_expiry)
    percentage_to_look_upto = std_to_look_upto * vol_till_expiry
    min_strike = atm_strike*(1 - percentage_to_look_upto*std_to_look_upto)
    max_strike = atm_strike*(1 + percentage_to_look_upto*std_to_look_upto)
    return min_strike, max_strike


def GetContractSpecsListFromYahooFin(symbol, calendar_dte_limit, std_to_look_upto):
    stock = yf.Ticker(symbol)
    expiry_list = stock.options
    contract_spec_list = []
    for expiry in expiry_list:
        expiry_date_time = parser.parse(expiry)
        if (expiry_date_time - dt.datetime.now()).days > calendar_dte_limit:
            continue
        option_chain = stock.option_chain(date = expiry)
        put_df = option_chain.puts
        put_df = put_df[put_df.inTheMoney == False]
        atm_row = put_df.loc[put_df['strike'].idxmax()]
        atm_volatility = atm_row.impliedVolatility
        atm_strike = atm_row.strike
        min_strike, max_strike = FindMinMaxStrike( expiry, atm_volatility, std_to_look_upto, atm_strike)
        for i, row in put_df.iterrows():
            strike = row['strike']
            if min_strike < strike < max_strike:
                contract_spec_dict = {'Symbol' : symbol, "Expiry" : expiry.replace("-",""), 'Strike' : strike, 'Type' : "P", 'Exchange' : "SMART"}
                contract_spec_list.append(contract_spec_dict)
        call_df = option_chain.calls
        call_df = call_df[call_df.inTheMoney == False]

        for i, row in call_df.iterrows():
            strike = row['strike']
            if min_strike < strike < max_strike:
                contract_spec_dict = {'Symbol' : symbol, "Expiry" : expiry.replace("-",""), 'Strike' : strike, 'Type' : "C", 'Exchange' : "SMART"}
                contract_spec_list.append(contract_spec_dict)
    return contract_spec_list

def GetCurrentPrice(contract):
    temp_contract= contract
    ib.qualifyContracts(temp_contract)
    [ticker] = ib.reqTickers(temp_contract)
    ib.sleep(1)
    return ticker.marketPrice()

In [6]:
# inputs_path = "./../../Inputs/historical_data_inputs.csv"
# data_path = "./../../HistoricalData"
# interval_time_list = ['1 day', '1 hour', '10 mins']
# inputs_dict = GetInputs(inputs_path)


In [7]:
symbol = 'AAPL'

In [8]:
contract_spec_list = GetContractSpecsListFromYahooFin(symbol, 20, 1.5)

In [9]:
contracts = []
for contract_spec in contract_spec_list:
    contract = Option(contract_spec['Symbol'], contract_spec['Expiry'], contract_spec['Strike'],contract_spec['Type'], contract_spec['Exchange'])
    contracts.append(contract)

In [10]:
contracts = ib.qualifyContracts(*contracts)
print("Total number of contracts : ", len(contracts))
tickers = ib.reqTickers(*contracts)
print("Sample Ticker : ", tickers[0])

Total number of contracts :  74
Sample Ticker :  Ticker(contract=Option(conId=586424433, symbol='AAPL', lastTradeDateOrContractMonth='20220930', strike=138.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='AAPL  220930P00138000', tradingClass='AAPL'), time=datetime.datetime(2022, 9, 27, 19, 6, 58, 174484, tzinfo=datetime.timezone.utc), bid=0.13, bidSize=4984.0, ask=0.14, askSize=4153.0, last=0.13, lastSize=1.0, volume=2955.0, high=0.24, low=0.09, close=0.26, halted=0.0, bidGreeks=OptionComputation(tickAttrib=0, impliedVol=0.5995664105156073, delta=-0.03901627221480768, optPrice=0.12999999523162842, pvDividend=0.0, gamma=0.010182513160454126, vega=0.012249066948213483, theta=-0.11481992371210853, undPrice=151.6699981689453), askGreeks=OptionComputation(tickAttrib=0, impliedVol=0.6077430085470688, delta=-0.04104575449425753, optPrice=0.14000000059604645, pvDividend=0.0, gamma=0.010467785944867247, vega=0.012251847180607622, theta=-0.12128015079424835, undPric

In [11]:
ticker_list= tickers

In [12]:


#creating raw dataframe from option tickers
und_price_dict = {}
row_list_for_df = []

for ticker in ticker_list:
    if ticker.modelGreeks == None:
        continue
    expiry = dt.datetime.strptime(ticker.contract.lastTradeDateOrContractMonth , '%Y%m%d').date()
    strike = ticker.contract.strike
    kind = ticker.contract.right
    if ticker.modelGreeks.undPrice == None:
        if symbol in  und_price_dict:
            und_price = und_price_dict[symbol]
        else:
            und_price = GetCurrentPrice(Stock(symbol, 'SMART', 'USD'))
            und_price_dict[symbol] = und_price
    else:
        und_price = ticker.modelGreeks.undPrice
        und_price_dict[symbol] = und_price
    bid_price = ticker.bid
    ask_price = ticker.ask
    mid_price = (bid_price+ask_price)/2
    last_price = ticker.last
    volume = ticker.volume
    ib_bid_delta = ticker.bidGreeks.delta
    data_collection_time = dt.datetime.now()
    ticker_dict_temp = {'data_collection_time' : data_collection_time, 'symbol' : symbol , 'expiry' : expiry, 'strike' : strike, 'kind': kind, 'und_price' : und_price,
                   'bid_price': bid_price, 'ask_price' : ask_price, 'mid_price' : mid_price,
                   'last_price' : last_price, 'volume' : volume, 'ib_bid_delta' : ib_bid_delta }
    row_list_for_df.append(ticker_dict_temp)
df = pd.DataFrame(row_list_for_df)


In [13]:
df['tte'] = df.apply(lambda x : trading_utils.tte(dt.datetime.combine(x.expiry, dt.datetime.min.time()), x.data_collection_time), axis = 1)
df['implied_vol_mid_point'] = df.apply(lambda x : trading_utils.bs_implied_vol(x.und_price, x.strike, x.tte, x.kind, x.mid_price), axis= 1)
df['mid_point_delta'] =  df.apply(lambda x : trading_utils.bs_delta(x.und_price, x.strike, x.tte, x.kind, x.implied_vol_mid_point), axis = 1)

#restructure df to get call and put in same row
df_calls = df[df.kind == 'C']
df_puts = df[df.kind == 'P']
single_df = pd.merge(df_calls, df_puts, on = ['symbol', 'expiry', 'strike'], suffixes = ("_call", "_put"))
single_df['und_price'] = single_df.und_price_call
single_df['tte'] = single_df.tte_call
single_df = single_df.drop(columns = ['kind_call', 'kind_put', 'und_price_put', 'tte_put', 'und_price_call', 'tte_call' ])



#restructure to get otm option
single_df['bid_price_otm'] = single_df.apply(lambda x : x.bid_price_call if x.mid_point_delta_call < 0.5 else x.bid_price_put, axis = 1)
single_df['ask_price_otm'] = single_df.apply(lambda x : x.ask_price_call if x.mid_point_delta_call < 0.5 else x.ask_price_put, axis = 1)
single_df['mid_price_otm'] = single_df.apply(lambda x : x.mid_price_call if x.mid_point_delta_call < 0.5 else x.mid_price_put, axis = 1)
single_df['implied_vol_mid_point_otm'] = single_df.apply(lambda x : x.implied_vol_mid_point_call if x.mid_point_delta_call < 0.5 else x.implied_vol_mid_point_put, axis = 1)
single_df['mid_point_delta_otm'] = single_df.apply(lambda x : x.mid_point_delta_call if x.mid_point_delta_call < 0.5 else x.mid_point_delta_put, axis = 1)
single_df['mid_point_call_delta_otm'] = single_df.apply(lambda x : x.mid_point_delta_otm if x.mid_point_delta_otm > 0 else (1 - x.mid_point_delta_otm*-1), axis = 1)


#filter out the columns
columns_to_filter = ['symbol', 'expiry', 'strike','und_price', 'tte', 'bid_price_otm',
       'ask_price_otm', 'mid_price_otm', 'implied_vol_mid_point_otm',
       'mid_point_delta_otm', 'mid_point_call_delta_otm']
single_df = single_df[columns_to_filter]
single_df = single_df.dropna()

C:\ProgramData\Anaconda3\lib\site-packages\py_vollib\ref_python\black_scholes\__init__.py:83: RuntimeWarning: invalid value encountered in sqrt
  denominator = sigma * numpy.sqrt(t)


ValueError: Wrong number of items passed 23, placement implies 1

In [13]:
contracts

[Option(symbol='AAPL', lastTradeDateOrContractMonth='20220930', strike=150, right='C', exchange='SMART'),
 Option(symbol='AAPL', lastTradeDateOrContractMonth='20220930', strike=155, right='C', exchange='SMART')]

In [14]:
contracts = ib.qualifyContracts(*contracts)
print("Total number of contracts : ", len(contracts))
tickers = ib.reqTickers(*contracts)
print("Sample Ticker : ", tickers[0])

Total number of contracts :  2
Sample Ticker :  Ticker(contract=Option(conId=578903604, symbol='AAPL', lastTradeDateOrContractMonth='20220930', strike=150.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='AAPL  220930C00150000', tradingClass='AAPL'), time=datetime.datetime(2022, 9, 26, 18, 54, 14, 959144, tzinfo=datetime.timezone.utc), bid=3.8, bidSize=11682.0, ask=3.9, askSize=7395.0, last=3.8, lastSize=1.0, volume=16382.0, high=5.3, low=2.98, close=3.52, bidGreeks=OptionComputation(tickAttrib=0, impliedVol=0.46316396248079406, delta=0.5985655801679204, optPrice=3.799999952316284, pvDividend=0.0, gamma=0.05262162691316894, vega=0.06057817020981737, theta=-0.3629789040013458, undPrice=151.60000610351562), askGreeks=OptionComputation(tickAttrib=0, impliedVol=0.47142021459690153, delta=0.5972362657753542, optPrice=3.9000000953674316, pvDividend=0.0, gamma=0.051760515547668466, vega=0.06058078692103175, theta=-0.3697155693890758, undPrice=151.60000610351562), 

In [1]:
ib.disconnect()

NameError: name 'ib' is not defined